In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, learning_curve, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import platform
import sys
import sklearn
from skopt import BayesSearchCV
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Categorical, Integer, Continuous
from sklearn.model_selection import KFold

# Load the dataset
data = pd.read_csv('E:/UOL/Semester 3/Dissertation/Dissertation Draft/final analysis/datasetfinal.csv')

# Remove the % sign and convert to float
data = data.replace('%', '', regex=True).astype(float)

# Split the dataset into features and target variable
X = data.iloc[:, 1:-1]
y = data.iloc[:, -1]
years = data.iloc[:, 0]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test, years_train, years_test = train_test_split(X, y, years, test_size=0.2, random_state=42)

# Initialize the models
models = {
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
    'SVM': SVR()
}

# Train the models and make predictions
predictions = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions[name] = model.predict(X_test)

# Calculate the metrics
metrics = {}
for name, y_pred in predictions.items():
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    n = len(y_test)
    p = X_test.shape[1]
    ar2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    rmse = np.sqrt(mse)
    mbd = np.mean(y_pred - y_test)
    metrics[name] = [mse, mae, r2, ar2, rmse, mbd]

# Save the metrics to a CSV file
metrics_df = pd.DataFrame(metrics, index=['MSE', 'MAE', 'R2', 'AR2', 'RMSE', 'MBD']).T
metrics_df.to_csv('metrics.csv')

# Generate and save plots
for name, model in models.items():
    y_pred = predictions[name]
    
    # Residual Plot
    residuals = y_test - y_pred
    plt.figure(figsize=(10, 6))
    sns.residplot(x=y_pred, y=residuals, lowess=True, color='green')
    plt.title(f'Residual Plot for {name}')
    plt.xlabel('Predicted')
    plt.ylabel('Residuals')
    plt.savefig(f'{name}_residual_plot.png', dpi=300)
    plt.close()

    # Prediction vs. Actual Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, color='blue')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
    plt.title(f'Prediction vs Actual for {name}')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.savefig(f'{name}_prediction_vs_actual.png', dpi=300)
    plt.close()

    # Error Distribution Plot
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True, color='red')
    plt.title(f'Error Distribution for {name}')
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.savefig(f'{name}_error_distribution.png', dpi=300)
    plt.close()

    # Learning Curves
    train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    train_scores_mean = -np.mean(train_scores, axis=1)
    test_scores_mean = -np.mean(test_scores, axis=1)
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_scores_mean, 'o-', color='green', label='Training error')
    plt.plot(train_sizes, test_scores_mean, 'o-', color='red', label='Cross-validation error')
    plt.title(f'Learning Curves for {name}')
    plt.xlabel('Training size')
    plt.ylabel('Error')
    plt.legend(loc='best')
    plt.savefig(f'{name}_learning_curve.png', dpi=300)
    plt.close()

    # Feature Importance Plot (only for tree-based models)
    if hasattr(model, 'feature_importances_'):
        plt.figure(figsize=(10, 6))
        feature_importances = model.feature_importances_
        features = X.columns
        indices = np.argsort(feature_importances)
        plt.barh(range(len(indices)), feature_importances[indices], color='magenta', align='center')
        plt.yticks(range(len(indices)), [features[i] for i in indices])
        plt.title(f'Feature Importance for {name}')
        plt.xlabel('Relative Importance')
        plt.savefig(f'{name}_feature_importance.png', dpi=300)
        plt.close()

# Save predictions to a CSV file
predictions_df = pd.DataFrame({'Year': years_test})
for name, y_pred in predictions.items():
    predictions_df[f'{name} Prediction'] = y_pred
predictions_df['Actual'] = y_test.values
predictions_df.to_csv('predictions.csv', index=False)

# Save training and testing datasets to CSV files
train_df = pd.concat([years_train, X_train, y_train], axis=1)
test_df = pd.concat([years_test, X_test, y_test], axis=1)
train_df.to_csv('training_data.csv', index=False)
test_df.to_csv('testing_data.csv', index=False)

print("Grid Search HPO Results\n")

#without SVM

# Environment and library versions
print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"Seaborn version: {sns.__version__}")
print(f"Platform: {platform.platform()}")

# Split the dataset into features and target variable
X = data.iloc[:, 1:-1]
y = data.iloc[:, -1]
years = data.iloc[:, 0]

# Define the parameter grids for each model
param_grids = {
    'Decision Tree': {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 10, 20]
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 10, 20]
    },
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 6, 9]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'epsilon': [0.1, 0.2, 0.3],
        'kernel': ['linear', 'rbf']
    }
}

# Initialize the models
models = {
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
    'SVM': SVR()
}

# Perform Grid Search and make predictions
gs_predictions = {}
best_params = {}
run_times = {}
for name, model in models.items():
    start_time = time.time()
    grid_search = GridSearchCV(estimator=model, param_grid=param_grids[name], cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    end_time = time.time()
    best_model = grid_search.best_estimator_
    gs_predictions[name] = best_model.predict(X_test)
    best_params[name] = grid_search.best_params_
    run_times[name] = end_time - start_time
    print(f"Best hyperparameters for {name}: {best_params[name]}")

# Calculate the metrics
metrics = {}
for name, y_pred in gs_predictions.items():
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    n = len(y_test)
    p = X_test.shape[1]
    ar2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    rmse = np.sqrt(mse)
    mbd = np.mean(y_pred - y_test)
    metrics[name] = [mse, mae, r2, ar2, rmse, mbd]

# Save the metrics to a CSV file
metrics_df = pd.DataFrame(metrics, index=['MSE', 'MAE', 'R2', 'AR2', 'RMSE', 'MBD']).T
metrics_df.to_csv('metrics_gsoptimized.csv')

# Generate and save plots
for name, model in models.items():
    y_pred = gs_predictions[name]
    
    # Residual Plot
    residuals = y_test - y_pred
    plt.figure(figsize=(10, 6))
    sns.residplot(x=y_pred, y=residuals, lowess=True, color='green')
    plt.title(f'Residual Plot for {name} (GSoptimized)')
    plt.xlabel('Predicted')
    plt.ylabel('Residuals')
    plt.savefig(f'{name}_residual_plot_gsoptimized.png', dpi=300)
    plt.close()

    # Prediction vs. Actual Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, color='blue')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
    plt.title(f'Prediction vs Actual for {name} (GSoptimized)')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.savefig(f'{name}_prediction_vs_actual_gsoptimized.png', dpi=300)
    plt.close()

    # Error Distribution Plot
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True, color='red')
    plt.title(f'Error Distribution for {name} (GSoptimized)')
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.savefig(f'{name}_error_distribution_gsoptimized.png', dpi=300)
    plt.close()

    # Learning Curves
    train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    train_scores_mean = -np.mean(train_scores, axis=1)
    test_scores_mean = -np.mean(test_scores, axis=1)
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_scores_mean, 'o-', color='green', label='Training error')
    plt.plot(train_sizes, test_scores_mean, 'o-', color='red', label='Cross-validation error')
    plt.title(f'Learning Curves for {name} (GSoptimized)')
    plt.xlabel('Training size')
    plt.ylabel('Error')
    plt.legend(loc='best')
    plt.savefig(f'{name}_learning_curve_gsoptimized.png', dpi=300)
    plt.close()

    # Feature Importance Plot (only for tree-based models)
    if hasattr(model, 'feature_importances_'):
        plt.figure(figsize=(10, 6))
        feature_importances = model.feature_importances_
        features = X.columns
        indices = np.argsort(feature_importances)
        plt.barh(range(len(indices)), feature_importances[indices], color='magenta', align='center')
        plt.yticks(range(len(indices)), [features[i] for i in indices])
        plt.title(f'Feature Importance for {name} (GSoptimized)')
        plt.xlabel('Relative Importance')
        plt.savefig(f'{name}_feature_importance_gsoptimized.png', dpi=300)
        plt.close()

# Save predictions to a CSV file
predictions_df = pd.DataFrame({'Year': years_test})
for name, y_pred in gs_predictions.items():
    predictions_df[f'{name} GSoptimized Prediction'] = y_pred
predictions_df['Actual'] = y_test.values
predictions_df.to_csv('predictions_gsoptimized.csv', index=False)

# Save training and testing datasets to CSV files
train_df = pd.concat([years_train, X_train, y_train], axis=1)
test_df = pd.concat([years_test, X_test, y_test], axis=1)
train_df.to_csv('training_data.csv', index=False)
test_df.to_csv('testing_data.csv', index=False)

# Print run-time analysis
for name, runtime in run_times.items():
    print(f"{name} - Training and Prediction Time: {runtime:.2f} seconds")

print("Random Search HPO Results\n")

#without SVM

# Define the parameter distributions for each model
param_distributions = {
    'Decision Tree': {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 10, 20]
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 10, 20]
    },
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 6, 9]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'epsilon': [0.1, 0.2, 0.3],
        'kernel': ['linear', 'rbf']
    }
}

# Initialize the models
models = {
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
    'SVM': SVR()
}

# Perform Random Search and make predictions
rs_predictions = {}
best_params = {}
run_times = {}
for name, model in models.items():
    start_time = time.time()
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions[name], n_iter=10, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)
    random_search.fit(X_train, y_train)
    end_time = time.time()
    best_model = random_search.best_estimator_
    rs_predictions[name] = best_model.predict(X_test)
    best_params[name] = random_search.best_params_
    run_times[name] = end_time - start_time
    print(f"Best hyperparameters for {name}: {best_params[name]}")

# Calculate the metrics
metrics = {}
for name, y_pred in rs_predictions.items():
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    n = len(y_test)
    p = X_test.shape[1]
    ar2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    rmse = np.sqrt(mse)
    mbd = np.mean(y_pred - y_test)
    metrics[name] = [mse, mae, r2, ar2, rmse, mbd]

# Save the metrics to a CSV file
metrics_df = pd.DataFrame(metrics, index=['MSE', 'MAE', 'R2', 'AR2', 'RMSE', 'MBD']).T
metrics_df.to_csv('metrics_rsoptimized.csv')

# Generate and save plots
for name, model in models.items():
    y_pred = rs_predictions[name]
    
    # Residual Plot
    residuals = y_test - y_pred
    plt.figure(figsize=(10, 6))
    sns.residplot(x=y_pred, y=residuals, lowess=True, color='green')
    plt.title(f'Residual Plot for {name} (RSoptimized)')
    plt.xlabel('Predicted')
    plt.ylabel('Residuals')
    plt.savefig(f'{name}_residual_plot_rsoptimized.png', dpi=300)
    plt.close()

    # Prediction vs. Actual Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, color='blue')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
    plt.title(f'Prediction vs Actual for {name} (RSoptimized)')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.savefig(f'{name}_prediction_vs_actual_rsoptimized.png', dpi=300)
    plt.close()

    # Error Distribution Plot
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True, color='red')
    plt.title(f'Error Distribution for {name} (RSoptimized)')
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.savefig(f'{name}_error_distribution_rsoptimized.png', dpi=300)
    plt.close()

    # Learning Curves
    train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    train_scores_mean = -np.mean(train_scores, axis=1)
    test_scores_mean = -np.mean(test_scores, axis=1)
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_scores_mean, 'o-', color='green', label='Training error')
    plt.plot(train_sizes, test_scores_mean, 'o-', color='red', label='Cross-validation error')
    plt.title(f'Learning Curves for {name} (RSoptimized)')
    plt.xlabel('Training size')
    plt.ylabel('Error')
    plt.legend(loc='best')
    plt.savefig(f'{name}_learning_curve_rsoptimized.png', dpi=300)
    plt.close()

    # Feature Importance Plot (only for tree-based models)
    if hasattr(model, 'feature_importances_'):
        plt.figure(figsize=(10, 6))
        feature_importances = model.feature_importances_
        features = X.columns
        indices = np.argsort(feature_importances)
        plt.barh(range(len(indices)), feature_importances[indices], color='magenta', align='center')
        plt.yticks(range(len(indices)), [features[i] for i in indices])
        plt.title(f'Feature Importance for {name} (RSoptimized)')
        plt.xlabel('Relative Importance')
        plt.savefig(f'{name}_feature_importance_rsoptimized.png', dpi=300)
        plt.close()

# Save predictions to a CSV file
predictions_df = pd.DataFrame({'Year': years_test})
for name, y_pred in rs_predictions.items():
    predictions_df[f'{name} RSoptimized Prediction'] = y_pred
predictions_df['Actual'] = y_test.values
predictions_df.to_csv('predictions_rsoptimized.csv', index=False)

# Save training and testing datasets to CSV files
train_df = pd.concat([years_train, X_train, y_train], axis=1)
test_df = pd.concat([years_test, X_test, y_test], axis=1)
train_df.to_csv('training_data.csv', index=False)
test_df.to_csv('testing_data.csv', index=False)

# Print run-time analysis
for name, runtime in run_times.items():
    print(f"{name} - Training and Prediction Time: {runtime:.2f} seconds")

print("Bayesian Optimization HPO Results\n")

#without SVM

# Define the parameter search spaces for each model
param_spaces = {
    'Decision Tree': {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 10, 20]
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 10, 20]
    },
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 6, 9]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'epsilon': [0.1, 0.2, 0.3],
        'kernel': ['linear', 'rbf']
    }
}

# Initialize the models
models = {
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
    'SVM': SVR()
}

# Perform Bayesian Optimization and make predictions
bo_predictions = {}
best_params = {}
run_times = {}
for name, model in models.items():
    start_time = time.time()
    bayes_search = BayesSearchCV(estimator=model, search_spaces=param_spaces[name], n_iter=30, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)
    bayes_search.fit(X_train, y_train)
    end_time = time.time()
    best_model = bayes_search.best_estimator_
    bo_predictions[name] = best_model.predict(X_test)
    best_params[name] = bayes_search.best_params_
    run_times[name] = end_time - start_time
    print(f"Best hyperparameters for {name}: {best_params[name]}")

# Calculate the metrics
metrics = {}
for name, y_pred in bo_predictions.items():
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    n = len(y_test)
    p = X_test.shape[1]
    ar2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    rmse = np.sqrt(mse)
    mbd = np.mean(y_pred - y_test)
    metrics[name] = [mse, mae, r2, ar2, rmse, mbd]

# Save the metrics to a CSV file
metrics_df = pd.DataFrame(metrics, index=['MSE', 'MAE', 'R2', 'AR2', 'RMSE', 'MBD']).T
metrics_df.to_csv('metrics_booptimized.csv')

# Generate and save plots
for name, model in models.items():
    y_pred = bo_predictions[name]
    
    # Residual Plot
    residuals = y_test - y_pred
    plt.figure(figsize=(10, 6))
    sns.residplot(x=y_pred, y=residuals, lowess=True, color='green')
    plt.title(f'Residual Plot for {name} (BOoptimized)')
    plt.xlabel('Predicted')
    plt.ylabel('Residuals')
    plt.savefig(f'{name}_residual_plot_booptimized.png', dpi=300)
    plt.close()

    # Prediction vs. Actual Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, color='blue')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
    plt.title(f'Prediction vs Actual for {name} (BOoptimized)')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.savefig(f'{name}_prediction_vs_actual_booptimized.png', dpi=300)
    plt.close()

    # Error Distribution Plot
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True, color='red')
    plt.title(f'Error Distribution for {name} (BOoptimized)')
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.savefig(f'{name}_error_distribution_booptimized.png', dpi=300)
    plt.close()

    # Learning Curves
    train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    train_scores_mean = -np.mean(train_scores, axis=1)
    test_scores_mean = -np.mean(test_scores, axis=1)
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_scores_mean, 'o-', color='green', label='Training error')
    plt.plot(train_sizes, test_scores_mean, 'o-', color='red', label='Cross-validation error')
    plt.title(f'Learning Curves for {name} (BOoptimized)')
    plt.xlabel('Training size')
    plt.ylabel('Error')
    plt.legend(loc='best')
    plt.savefig(f'{name}_learning_curve_booptimized.png', dpi=300)
    plt.close()

    # Feature Importance Plot (only for tree-based models)
    if hasattr(model, 'feature_importances_'):
        plt.figure(figsize=(10, 6))
        feature_importances = model.feature_importances_
        features = X.columns
        indices = np.argsort(feature_importances)
        plt.barh(range(len(indices)), feature_importances[indices], color='magenta', align='center')
        plt.yticks(range(len(indices)), [features[i] for i in indices])
        plt.title(f'Feature Importance for {name} (BOoptimized)')
        plt.xlabel('Relative Importance')
        plt.savefig(f'{name}_feature_importance_booptimized.png', dpi=300)
        plt.close()

# Save predictions to a CSV file
predictions_df = pd.DataFrame({'Year': years_test})
for name, y_pred in bo_predictions.items():
    predictions_df[f'{name} BOoptimized Prediction'] = y_pred
predictions_df['Actual'] = y_test.values
predictions_df.to_csv('predictions_booptimized.csv', index=False)

# Save training and testing datasets to CSV files
train_df = pd.concat([years_train, X_train, y_train], axis=1)
test_df = pd.concat([years_test, X_test, y_test], axis=1)
train_df.to_csv('training_data.csv', index=False)
test_df.to_csv('testing_data.csv', index=False)

# Print run-time analysis
for name, runtime in run_times.items():
    print(f"{name} - Training and Prediction Time: {runtime:.2f} seconds")

print("K-Fold Cross-Validation HPO Results\n")

#without SVM

# Define the parameter grids for each model
param_grids = {
    'Decision Tree': {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 10, 20]
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 10, 20]
    },
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 6, 9]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'epsilon': [0.1, 0.2, 0.3],
        'kernel': ['linear', 'rbf']
    }
}

# Initialize the models
models = {
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBoost': XGBRegressor(),
    'SVM': SVR()
}

# Perform Grid Search with K-Fold Cross-Validation and make predictions
cv_predictions = {}
best_params = {}
run_times = {}
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_results = {}

for name, model in models.items():
    start_time = time.time()
    grid_search = GridSearchCV(estimator=model, param_grid=param_grids[name], cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    end_time = time.time()
    best_model = grid_search.best_estimator_
    cv_predictions[name] = best_model.predict(X_test)
    best_params[name] = grid_search.best_params_
    run_times[name] = end_time - start_time
    fold_results[name] = cross_val_score(best_model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')
    print(f"Best hyperparameters for {name}: {best_params[name]}")

# Calculate the metrics
metrics = {}
for name, y_pred in cv_predictions.items():
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    n = len(y_test)
    p = X_test.shape[1]
    ar2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    rmse = np.sqrt(mse)
    mbd = np.mean(y_pred - y_test)
    metrics[name] = [mse, mae, r2, ar2, rmse, mbd]

# Save the metrics to a CSV file
metrics_df = pd.DataFrame(metrics, index=['MSE', 'MAE', 'R2', 'AR2', 'RMSE', 'MBD']).T
metrics_df.to_csv('metrics_cvoptimized.csv')

# Generate and save plots
for name, model in models.items():
    y_pred = cv_predictions[name]
    
    # Residual Plot
    residuals = y_test - y_pred
    plt.figure(figsize=(10, 6))
    sns.residplot(x=y_pred, y=residuals, lowess=True, color='green')
    plt.title(f'Residual Plot for {name} (CVoptimized)')
    plt.xlabel('Predicted')
    plt.ylabel('Residuals')
    plt.savefig(f'{name}_residual_plot_cvoptimized.png', dpi=300)
    plt.close()

    # Prediction vs. Actual Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, color='blue')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
    plt.title(f'Prediction vs Actual for {name} (CVoptimized)')
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.savefig(f'{name}_prediction_vs_actual_cvoptimized.png', dpi=300)
    plt.close()

    # Error Distribution Plot
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True, color='red')
    plt.title(f'Error Distribution for {name} (CVoptimized)')
    plt.xlabel('Residuals')
    plt.ylabel('Frequency')
    plt.savefig(f'{name}_error_distribution_cvoptimized.png', dpi=300)
    plt.close()

    # Learning Curves
    train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
    train_scores_mean = -np.mean(train_scores, axis=1)
    test_scores_mean = -np.mean(test_scores, axis=1)
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_scores_mean, 'o-', color='green', label='Training error')
    plt.plot(train_sizes, test_scores_mean, 'o-', color='red', label='Cross-validation error')
    plt.title(f'Learning Curves for {name} (CVoptimized)')
    plt.xlabel('Training size')
    plt.ylabel('Error')
    plt.legend(loc='best')
    plt.savefig(f'{name}_learning_curve_cvoptimized.png', dpi=300)
    plt.close()

    # Feature Importance Plot (only for tree-based models)
    if hasattr(model, 'feature_importances_'):
        plt.figure(figsize=(10, 6))
        feature_importances = model.feature_importances_
        features = X.columns
        indices = np.argsort(feature_importances)
        plt.barh(range(len(indices)), feature_importances[indices], color='magenta', align='center')
        plt.yticks(range(len(indices)), [features[i] for i in indices])
        plt.title(f'Feature Importance for {name} (CVoptimized)')
        plt.xlabel('Relative Importance')
        plt.savefig(f'{name}_feature_importance_cvoptimized.png', dpi=300)
        plt.close()

# Save predictions to a CSV file
predictions_df = pd.DataFrame({'Year': years_test})
for name, y_pred in cv_predictions.items():
    predictions_df[f'{name} CVoptimized Prediction'] = y_pred
predictions_df['Actual'] = y_test.values
predictions_df.to_csv('predictions_cvoptimized.csv', index=False)

# Save training and testing datasets to CSV files
train_df = pd.concat([years_train, X_train, y_train], axis=1)
test_df = pd.concat([years_test, X_test, y_test], axis=1)
train_df.to_csv('training_data.csv', index=False)
test_df.to_csv('testing_data.csv', index=False)

# Print run-time analysis
for name, runtime in run_times.items():
    print(f"{name} - Training and Prediction Time: {runtime:.2f} seconds")

# Fold analysis
fold_analysis_df = pd.DataFrame(fold_results)
fold_analysis_df.to_csv('fold_analysis.csv')
print(fold_analysis_df)

# Plot fold analysis
plt.figure(figsize=(14, 7))
sns.boxplot(data=fold_analysis_df)
plt.title('Fold Analysis for Cross-Validation')
plt.xlabel('Model')
plt.ylabel('Negative Mean Squared Error')
plt.savefig('fold_analysis_boxplot.png', dpi=300)
plt.show()

# Cross-validation results table
cv_table = pd.DataFrame(fold_results).T
cv_table.columns = [f'Fold {i+1}' for i in range(cv_table.shape[1])]
cv_table['Mean'] = cv_table.mean(axis=1)
cv_table['Std'] = cv_table.std(axis=1)
print(cv_table)

# Plot cross-validation results table
plt.figure(figsize=(14, 7))
sns.heatmap(cv_table, annot=True, fmt=".2f", cmap="YlGnBu")
plt.title('Cross-Validation Results Table')
plt.show()

K-Fold Cross-Validation HPO Results

Best hyperparameters for Decision Tree: {'max_depth': 30, 'min_samples_split': 2}
Best hyperparameters for Random Forest: {'max_depth': 30, 'min_samples_split': 2, 'n_estimators': 100}
Best hyperparameters for XGBoost: {'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 100}


KeyboardInterrupt: 